#### 取对数（启用版）

In [1]:
import pandas as pd
import numpy as np
import json
import os

# 路径配置
# 数据来源：AeroSCOPE 2019 原始 CSV
AEROSCOPE_PATH = "../../data_origin/AeroSCOPE_Flights/AeroSCOPE_global_aviation_traffic_dataset_16_11.csv"
# 国家列表（你的 49 国子集，已包含 iso_2 字段）
JSON_PATH = "../country_list.json"
# 输出路径
OUT_FILE = "../../data/1-5-AeroSCOPE_Flight_Intensity.csv"

def get_flight_intensity_matrix(data_path: str, meta_path: str) -> pd.DataFrame:
    """
    基于航班频率计算归一化的国家间航空连接强度矩阵。
    逻辑：使用 iso_2 匹配原始数据，最后输出 iso (3位) 的矩阵。
    """
    # 1. 加载 49 国名单和映射
    with open(meta_path, 'r', encoding='utf-8') as f:
        countries = json.load(f)['countries']
        
        # 提取用于过滤的 2位代码 (对应 AeroSCOPE 数据)
        # 注意：JSON 里的 iso_2 可能带有空格，建议 strip() 清理一下
        target_iso2 = [c['iso_2'].strip() for c in countries if c.get('iso_2')]
        
        # 建立 2位 -> 3位 的映射字典，用于最后改名
        # Key: 'CN', Value: 'CHN'
        iso2_to_iso3 = {c['iso_2'].strip(): c['iso'] for c in countries if c.get('iso_2')}

    # 2. 读取 AeroSCOPE 数据
    # 只提取必要的列
    df = pd.read_csv(data_path, usecols=['departure_country', 'arrival_country', 'n_flights', 'domestic'])
    
    # 3. 过滤数据
    # 只保留 49 国之间的国际航线 (domestic == 0)
    # 使用 iso_2 列表进行筛选
    mask = (df['domestic'] == 0) & \
           (df['departure_country'].isin(target_iso2)) & \
           (df['arrival_country'].isin(target_iso2))
    df = df[mask]
    
    # 4. 构建 OD 矩阵 (按国家聚合航班数)
    # 此时行和列的索引是 2位代码 (如 CN, US)
    m = df.groupby(['departure_country', 'arrival_country'])['n_flights'].sum().unstack(fill_value=0)
    
    # 确保矩阵是 49x49 的完整矩阵，缺失的国家填充 0
    # 使用 target_iso2 进行重索引
    m = m.reindex(index=target_iso2, columns=target_iso2, fill_value=0)
    
    # 5. 对称化处理 (无向网络：A<->B 的连接强度)
    m = m + m.T
    
    # 6. 对数转换 (Log Transformation)
    m_log = np.log1p(m)
    
    # 7. 0-1 归一化 (Local Normalization)
    mask_diag = ~np.eye(len(target_iso2), dtype=bool)
    v_min = m_log.values[mask_diag].min()
    v_max = m_log.values[mask_diag].max()
    
    # 防止最大值最小值相等导致除以0
    if v_max != v_min:
        m_norm = (m_log - v_min) / (v_max - v_min)
    else:
        m_norm = m_log * 0

    # 将对角线置为 0
    np.fill_diagonal(m_norm.values, 0)
    
    # 8. 关键步骤：将索引从 ISO2 改回 ISO3
    # 这样你的输出文件就是标准的 3位代码，方便和其他数据对齐
    m_norm = m_norm.rename(index=iso2_to_iso3, columns=iso2_to_iso3)
    
    # 9. 按 ISO3 字母顺序排序并返回
    return m_norm.sort_index(axis=0).sort_index(axis=1)

if __name__ == "__main__":
    # 创建输出目录
    os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)
    
    print("正在处理 AeroSCOPE 航空连接数据...")
    if os.path.exists(AEROSCOPE_PATH):
        try:
            final_res = get_flight_intensity_matrix(AEROSCOPE_PATH, JSON_PATH)
            
            # 保存结果
            final_res.to_csv(OUT_FILE)
            print(f"处理完成！归一化矩阵已保存至: {OUT_FILE}")
            print(f"矩阵维度: {final_res.shape}")
            print("前5行预览：")
            print(final_res.iloc[:5, :5])
        except Exception as e:
            print(f"处理过程中发生错误: {e}")
    else:
        print(f"错误：找不到原始数据文件 {AEROSCOPE_PATH}")

正在处理 AeroSCOPE 航空连接数据...
处理完成！归一化矩阵已保存至: ../../data/1-5-AeroSCOPE_Flight_Intensity.csv
矩阵维度: (49, 49)
前5行预览：
arrival_country    ARG  AUS       AUT       BEL       BGR
departure_country                                        
ARG                0.0  0.0  0.000000  0.000000  0.000000
AUS                0.0  0.0  0.000000  0.000000  0.000000
AUT                0.0  0.0  0.000000  0.680557  0.671199
BEL                0.0  0.0  0.680557  0.000000  0.608910
BGR                0.0  0.0  0.671199  0.608910  0.000000


##### 不取对数

In [2]:
import pandas as pd
import numpy as np
import json
import os

# 路径配置
# 数据来源：AeroSCOPE 2019 原始 CSV
AEROSCOPE_PATH = "../../data_origin/AeroSCOPE_Flights/AeroSCOPE_global_aviation_traffic_dataset_16_11.csv"
# 国家列表（你的 49 国子集，已包含 iso_2 字段）
JSON_PATH = "../country_list.json"
# 输出路径
OUT_FILE = "../../data/1-5-AeroSCOPE_Flight_Intensity.csv"

def get_flight_intensity_matrix(data_path: str, meta_path: str) -> pd.DataFrame:
    """
    基于航班频率计算归一化的国家间航空连接强度矩阵。
    逻辑：使用 iso_2 匹配原始数据，最后输出 iso (3位) 的矩阵。
    """
    # 1. 加载 49 国名单和映射
    with open(meta_path, 'r', encoding='utf-8') as f:
        countries = json.load(f)['countries']
        
        # 提取用于过滤的 2位代码 (对应 AeroSCOPE 数据)
        # 注意：JSON 里的 iso_2 可能带有空格，建议 strip() 清理一下
        target_iso2 = [c['iso_2'].strip() for c in countries if c.get('iso_2')]
        
        # 建立 2位 -> 3位 的映射字典，用于最后改名
        # Key: 'CN', Value: 'CHN'
        iso2_to_iso3 = {c['iso_2'].strip(): c['iso'] for c in countries if c.get('iso_2')}

    # 2. 读取 AeroSCOPE 数据
    # 只提取必要的列
    df = pd.read_csv(data_path, usecols=['departure_country', 'arrival_country', 'n_flights', 'domestic'])
    
    # 3. 过滤数据
    # 只保留 49 国之间的国际航线 (domestic == 0)
    # 使用 iso_2 列表进行筛选
    mask = (df['domestic'] == 0) & \
           (df['departure_country'].isin(target_iso2)) & \
           (df['arrival_country'].isin(target_iso2))
    df = df[mask]
    
    # 4. 构建 OD 矩阵 (按国家聚合航班数)
    # 此时行和列的索引是 2位代码 (如 CN, US)
    m = df.groupby(['departure_country', 'arrival_country'])['n_flights'].sum().unstack(fill_value=0)
    
    # 确保矩阵是 49x49 的完整矩阵，缺失的国家填充 0
    # 使用 target_iso2 进行重索引
    m = m.reindex(index=target_iso2, columns=target_iso2, fill_value=0)
    
    # 5. 对称化处理 (无向网络：A<->B 的连接强度)
    m = m + m.T
    
    # 6. 对数转换 (Log Transformation)
    # === 修改处：去掉 np.log1p ===
    m_log = m
    
    # 7. 0-1 归一化 (Local Normalization)
    mask_diag = ~np.eye(len(target_iso2), dtype=bool)
    v_min = m_log.values[mask_diag].min()
    v_max = m_log.values[mask_diag].max()
    
    # 防止最大值最小值相等导致除以0
    if v_max != v_min:
        m_norm = (m_log - v_min) / (v_max - v_min)
    else:
        m_norm = m_log * 0

    # 将对角线置为 0
    np.fill_diagonal(m_norm.values, 0)
    
    # 8. 关键步骤：将索引从 ISO2 改回 ISO3
    # 这样你的输出文件就是标准的 3位代码，方便和其他数据对齐
    m_norm = m_norm.rename(index=iso2_to_iso3, columns=iso2_to_iso3)
    
    # 9. 按 ISO3 字母顺序排序并返回
    return m_norm.sort_index(axis=0).sort_index(axis=1)

if __name__ == "__main__":
    # 创建输出目录
    os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)
    
    print("正在处理 AeroSCOPE 航空连接数据...")
    if os.path.exists(AEROSCOPE_PATH):
        try:
            final_res = get_flight_intensity_matrix(AEROSCOPE_PATH, JSON_PATH)
            
            # 保存结果
            final_res.to_csv(OUT_FILE)
            print(f"处理完成！归一化矩阵已保存至: {OUT_FILE}")
            print(f"矩阵维度: {final_res.shape}")
            print("前5行预览：")
            print(final_res.iloc[:5, :5])
        except Exception as e:
            print(f"处理过程中发生错误: {e}")
    else:
        print(f"错误：找不到原始数据文件 {AEROSCOPE_PATH}")

正在处理 AeroSCOPE 航空连接数据...
处理完成！归一化矩阵已保存至: ../../data/1-5-AeroSCOPE_Flight_Intensity.csv
矩阵维度: (49, 49)
前5行预览：
arrival_country    ARG  AUS       AUT       BEL       BGR
departure_country                                        
ARG                0.0  0.0  0.000000  0.000000  0.000000
AUS                0.0  0.0  0.000000  0.000000  0.000000
AUT                0.0  0.0  0.000000  0.018532  0.016488
BEL                0.0  0.0  0.018532  0.000000  0.007574
BGR                0.0  0.0  0.016488  0.007574  0.000000
